# 数据抽象

数据抽象简单讲，就是讲处理复杂数据的部分跟复杂数据如何表示隔离开，  

文中一开始的例子是有理数计算
作为开始，假设已经有一种从分子和分母构造有理数的方法，并进一步假定，如果有一个有理数，我们可以有一种方法取得他们的分子和分母。

- (make-rat \<n> \<d>) : 返回一个有理数，分子为n，分母为d
- (number x) : 返回有理数x的分子
- (denom x)  : 返回有理数x的分母

我们这里使用一种称为 **按愿望思维**的策略，现在我们并没有说有理数将如何表示，也没有说number等如何实现，这里是假设已经有了这几个过程。然后我们就可以做有理数的加减程序和判断了

$\begin{equation}\begin{array}{l}
& \frac{\Huge{n_{1}}}{\Huge{d_1}}+\frac{\Huge{n_2}}{\Huge{d_2}}= \frac{\Huge{n_1d_2+n_2d_1}}{\Huge{d_1d_2}}\\
& \Huge{\frac{n_1}{d_1}-\frac{n_2}{d_2}= \frac{n_1d_2-n_2d_1}{d_1d_2}}\\
& \Huge{\frac{n_1}{d_1}\cdot\frac{n_2}{d_2}=\frac{n_1n_2}{d_1d_2}}\\
& \Huge{\frac{n_1/d_2}{n_2/d_2}=\frac{n_1d_2}{d_1n_2}}\\
& \Huge{\frac{n_1}{d_1}=\frac{n_2}{d_2} 当且仅当  n_1d_2 = n_2d_1}
\end{array}\end{equation}$

In [1]:
(define (add-rat x y)
  (make-rat (+ (* (numer x) (denom y))
               (* (numer y) (denom x)))
            (* (denom x) (denom y))))

(define (sub-rat x y)
  (make-rat (- (* (numer x) (denom y))
               (* (numer y) (denom x)))
            (* (denom x) (denom y))))

(define (mul-rat x y)
  (make-rat (* (numer x) (numer y))
            (* (denom x) (denom y))))

(define (div-rat x y)
  (make-rat (* (numer x) (denom y))
            (* (denom x) (numer y))))

(define (equal-rat? x y)
  (= (* (numer x) (denom y))
     (* (numer y) (denom x))))

; 这样我们已经有了在选择和构造的基础上各种有理数的计算，而这时候我们的有理数还没有定义呢。
; 现在我们需要有某种形式，将一个分子和分母粘贴起来，构造成有理数》
; 这里用lisp的序对来构造有理数

(define (make-rat n d) (cons n d))
(define (numer x) (car x))
(define (denom x) (cdr x))

; 这里为了显示计算结果，显示打印吧

(define (print-rat x)
  (newline)
  (display (numer x))
  (display "/")
  (display (denom x)))

; 如下是测试一下
(define one-half (make-rat 1 2))            ; 顶一个一个有理数
(define one-third (make-rat 1 3))           ; 定义另一个有理数。

In [2]:
(print-rat one-half)                        ; 打印有理数


1/2

In [3]:
(print-rat (add-rat one-half one-third))    ; 有理数加法


5/6

In [4]:
(print-rat (mul-rat one-half one-third))    ; 有理数乘法


1/6

In [5]:
(print-rat (add-rat one-third one-third))


6/9

In [6]:
; 可以看到上边的这个没有将有理数化简为最简形式

(define (gcd a b)                     ; 求 ab的最大公约数
  (if (= b 0)
      a
      (gcd b (remainder a b))))

(define (make-rat n d)
  (let ((g (gcd n d)))                ; 这里是求最大公约数。
    (cons (/ n g) (/ d g))))          ; 然后分子分母都除以最大公约数就是最简形式了

(print-rat (add-rat one-third one-third))


2/3

抽象屏障，隔离了系统中的不同层次，在每一层上，这种屏障都把使用数据抽象的程序（上面）于实现数据抽象的程序（下面）分开来。

![](./图片/2-1.png)

In [7]:
; 练习2.4的
; 我这里前面加上my，不跟原先的冲突吧。
(define (mycons a b)
  (lambda (m) (m a b))) 
 
(define (mycar z) 
  (z (lambda (p q) p))) 

; 这里代换模型是这样子的。
; 假设如下表达式
(mycar (mycons 'x 'y)) 
(mycar (lambda (m) (m 'x 'y)))                ; 首先展开cons，
((lambda (m) (m 'x 'y)) (lambda (p q) p))     ; 将(lambda (m) (m x y))作为参数展开car
((lambda (p q) p) 'x 'y)                      ; 将(lambda (p q) p)作为m带入公式
; 将x作为p，y作为q带入公式。
; 最后得到x

; 他的答案就很简单啦
(define (mycdr z) 
  (z (lambda (p q) q))) 

(mycar (mycons 'x 'y)) 

'x

In [8]:
; 练习2.6
(define zero (lambda (f) (lambda (x) x)))    ; 其实就是这个有2个参数，取第二个参数的值。
((zero 4) 3)
; 这个将zero展开，就代换成
; ((lambda (x) x) 3) 
; 然后将3带入x就是最后的值

(define (add-1 n)
  (lambda (f) 
    (lambda (x) (f ((n f) x)))))

; 这里试试Zero加1是什么？
(add-1 zero)
;
(add-1 (lambda (f)
           (lambda (x)
               x)))

((lambda (n)                    ; add-1 展开
     (lambda (f)
         (lambda (x)
             (f ((n f) x)))))
 (lambda (f)                    ; zero
     (lambda (x)
         x)))
; 下边是带入n
(lambda (f) 
    (lambda (x)
        (f (
            ((lambda (f)        ; zero
                 (lambda (x)
                     x))
             f)
            x))))

(lambda (f)
    (lambda (x)
        (f ((lambda (x) x)
            x))))

(lambda (f)
    (lambda (x)
        (f x)))

; 经过展开得出 one 的定义为
(define one
    (lambda (f)
        (lambda (x)
            (f x))))

; 下边表示two
(add-1 one)

(add-1 (lambda (f)
           (lambda (x)
               (f x))))

((lambda (n)                    ; add-1
     (lambda (f)
         (lambda (x)
             (f ((n f) x)))))
 (lambda (f)                    ; one
     (lambda (x)
         (f x))))

(lambda (f)
    (lambda (x)
        (f ((
             (lambda (f)        ; one
                 (lambda (x)
                     (f x)))
             f)
            x))))

(lambda (f)
    (lambda (x)
        (f ((lambda (x)
                (f x))
            x))))

(lambda (f)
    (lambda (x)
        (f (f x))))

(define two
    (lambda (f)
        (lambda (x)
            (f (f x)))))

; 加法函数
; 通过对比 zero 、 one 和 two 的定义，我们可以发现，它们都接受两个参数 f 和 x ，不同的地方在于函数体内调用 f 的次数：

(define zero
    (lambda (f)
        (lambda (x)
            x)))            ; 没有 f

(define one
    (lambda (f)
        (lambda (x)
            (f x))))        ; 一个 f 调用

(define two
    (lambda (f)
        (lambda (x)
            (f (f x)))))    ; 两个 f 调用

; 因此，我们有理由相信， three 和 four 的定义很可能是：

(define three
    (lambda (f)
        (lambda (x)
            (f (f (f x))))))        ; 三个 f 调用

(define four
    (lambda (f)
        (lambda (x)
            (f (f (f (f x)))))))    ; 四个 f 调用

; 比如说， (+ 3 2) 的计算过程可以展开为：
;; (+ 3 2)

;; (+ (lambda (f)
;;        (lambda (x)
;;            (f (f (f x)))))
;;    (lambda (f)
;;        (lambda (x)
;;            (f (f x)))))

; ...

(lambda (f)
    (lambda (x)
        (f (f (f (f (f x)))))))

; 根据这个规则，可以写出相应的 Church 计数的加法函数：
(define my+
    (lambda (m)
        (lambda (n)
            (lambda (f)
                (lambda (x)
                    (m f (n f x)))))))

; 加法函数接受两个参数 m 和 n ，然后返回一个接受两个参数 f 和 x 的函数，
; 加法函数的函数体内， n 的函数体被表达式 (n f x) 取了出来，
; 然后又在表达式 (m f (n f x)) 中作为函数 m 的第二个函数被调用，
; 从而将 m 和 n 函数体内的 f 调用累积起来（如果有的话），从而形成加法效果。

In [9]:
; 练习 2.7 给定非空表里的最后一个元素
; 其实就是看看cdr是否为null吧
(define (last-pair x)
  (if (null? (cdr x))
      (car x)
      (last-pair (cdr x))))

(last-pair (list 1 2 3 4 5))

5

In [10]:
; 练习2.18 ，颠倒表的顺序。
(define (reverse lst)
    (iter lst '()))

(define (iter remained-items result)         ; 这个是迭代版本，result 来表示结果。
    (if (null? remained-items)
        result
        (iter (cdr remained-items)                  ; 这里是每次迭代剩余的
              (cons (car remained-items) result)))) ; 而这个是每次迭代保存的结果。
; 其实就是(car remained-items)在前面，表示每次都加到前面的意思。

(reverse (list 1 2 3 4 5))

'(5 4 3 2 1)

## 对表的操作

我们可以建立一个高级过程，将某一个过程应用于表的所有元素

In [11]:
(define (mymap proc items)  ; mymap不想跟map冲名而已，proc是过程，items是表
  (if (null? items)
      nil 
      (cons (proc (car items))
            (map proc (cdr items))))) 

(mymap abs (list -1 -2 -3 -4))   ; 这样就建立了一个处理表的高级抽象。

'(1 2 3 4)

In [12]:
; 练习 2.27 ，修改2.18所作的reverse过程，得到deep-reverse过程，
; 它以一个表作为参数，返回另一个表作为值，结果将表中的元素反转过来，其中的子树也反转。
; 其实就是做几个判断，
; 一个是判断是否是空树
; 一个是判断是否是叶子
; 其他
; https://sicp.readthedocs.io/en/latest/chp2/27.html 这个答案是有问题的，只是二叉树。
; 如下的支持任意树形结构。  https://blog.csdn.net/keyboardOTA/article/details/39910345
(define (deep-reverse input-list)
  (if (null? input-list)
      '()
      (append (deep-reverse (cdr input-list))          ; 这里就是顺序颠倒啦。
	      (if (list? (car input-list))                 ; 判断第一个是否为列表
			 (list (deep-reverse (car input-list)))    ; 则对列表递归调用。
			 (list (car input-list))))))

(define x (list (list 1 2 6) (list 3 4 5)))
(deep-reverse x)

'((5 4 3) (6 2 1))

In [13]:
; 练习 2.28 ，写一个过程fringe，以一个树为参数，返回一个表，表中的元素是这棵树的所有树叶，按照从左到右边的顺序。
; 树状结构的遍历

(define (fringe input-list)
  (if (null? input-list)
      '()
      (append 
       (if 
        (list? (car input-list))       ; 判断是否有根节点
		  (fringe (car input-list))    ; 有根节点，递归
		  (list (car input-list)))     ; 没根节点，只是第一项啦
	      (fringe (cdr input-list))))) ; 其他项目递归

(fringe x)

'(1 2 6 3 4 5)

In [14]:
; 练习 2.42 8皇后 


; 可以实现累加的。op为操作函数，
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op initial (cdr sequence)))))
; 比如如下的这个是累加。
(display (accumulate + 0 (list 1 2 3 4 5)))
(newline)
; 这个是累乘
(display (accumulate * 1 (list 1 2 3 4 5)))
(newline)

; 这个是展平的操作
(define (flatmap proc seq)
  (accumulate append nil (map proc seq)))
(define (inc x) (+ x 1))
(display (flatmap inc x))



(define (queens board-size)
  (define (queen-cols k)                    ; 定义一个内部过程
    (if (= k 0)                             ; 如果k为0，
        (list empty-board)                  ; 就返回空棋盘
        (filter                             ; 否则过滤，
         (lambda (positions) (safe? k positions))   ; 过滤条件是安全的点。
         (flatmap                                   ; 展平
          (lambda (rest-of-queens)
            (map (lambda (new-row)
                   (adjoin-position
                    new-row k rest-of-queens))
                 (enumerate-interval 1 board-size)))
          (queen-cols (- k 1))))))
  (queen-cols board-size))

15
120


nil: undefined;
 cannot reference an identifier before its definition
  in module: top-level


# 图形语言

描述一门语言时，应该将注意力集中到如下几点：
- 基本原语
- 组合手段
- 抽象手段

如下以一个图形语言为例子说明这个
- 语言只有一个元素：
   - ![](./图片/图形语言基本元素1.png)
- 组合方法 
   - beside 是组合2个画像成左右两边的
   - below  是组合2个画像成上下两边的
   - flip-vert  是返回一个上下颠倒的画像
   - flip-horiz 是返回一个左右颠倒的画像
   - ![](./图片/图形语言组合方法.png)
- 抽象方法
   - 比如如上的这个组合方法，用抽象方法是

# 数据导向
最关键的想法是通过显式处理操作-类型表格的方式，管理程序中的各种通用类型操作。

基于类型进行分派的方式，让每个操作管理自己的分派，从效果上看，这种方式就是将操作-类型表格分解为一行一行，每个通用性过程表示表格中的一行。

![](./图片/数据导向表1.png)

In [15]:
; (put <op> <type> <item>)
;  (get <op> <type>)

; 
(define (install-rectangular-package)
 ;; internal procedures
 (define (real-part z) (car z))
 (define (imag-part z) (cdr z))
 (define (make-from-real-imag x y) (cons x y))
 (define (magnitude z)
 (sqrt (+ (square (real-part z))
 (square (imag-part z)))))
 (define (angle z)
 (atan (imag-part z) (real-part z)))
 (define (make-from-mag-ang r a)
 (cons (* r (cos a)) (* r (sin a))))
 ;; interface to the rest of the system
 (define (tag x) (attach-tag 'rectangular x))
 (put 'real-part '(rectangular) real-part)
 (put 'imag-part '(rectangular) imag-part)
 (put 'magnitude '(rectangular) magnitude)
 (put 'angle '(rectangular) angle)
 (put 'make-from-real-imag 'rectangular
 (lambda (x y) (tag (make-from-real-imag x y))))
 (put 'make-from-mag-ang 'rectangular
 (lambda (r a) (tag (make-from-mag-ang r a))))
 'done)

; 选择函数，将通用性操作应用于一些参数。
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types -- APPLY-GENERIC"
           (list op type-tags))))))

; 各种通用型函数定义如下
(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

; 如下是定义复数
; 可以看到，定义也是从那个二维表中取得相应的操作的。
(define (make-from-real-imag x y)
 ((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a)
 ((get 'make-from-mag-ang 'polar) r a))

# 消息传递
采用“智能数据对象”，让它们基于操作名完成所需的分派任务。

消息传递将数据对象设想成一个实体，它以“消息”的方式接收到所选操作的名字。

In [16]:
(define (make-from-real-imag x y)      ; 这里就是一个智能对象，里边有相关的操作。
  (define (dispatch op)
    (cond ((eq? op 'real-part) x)
          ((eq? op 'imag-part) y)
          ((eq? op 'magnitude)
           (sqrt (+ (square x) (square y))))
          ((eq? op 'angle) (atan y x))
          (else
           (error "Unknown op -- MAKE-FROM-REAL-IMAG" op))))
  dispatch)   ; 清注意这个过程返回的是一个过程。

; 分派过程由如下的做
(define (apply-generic op arg) (arg op))

; 这个消息传递就是面向对象中的对象吧。然后各个操作相当于对象的属性。

# 数据导向和消息传递区别

通用型过程，其实都是一个二维表，区别在于
- 数据导向是按照行分隔，每个操作管理自己的分派。
- 消息传递是按照列分隔，是构建一个智能数据对象，每个智能对象里有各个操作。

在通用型过程上增加操作：
- 数据导向，在多个类型上都put一个函数就可以了。
- 消息传递，每个智能管理对象修改，增加一个操作。

在通用型过程上增加类型：
- 数据导向，为这个类型增加多有的操作。
- 消息传递，增加一个智能管理对象。

# 引用

>[SICP 解题集](https://sicp.readthedocs.io/en/latest/index.html)
>[SICP 习题 （2.27）解题总结：树状列表的遍历](https://blog.csdn.net/keyboardOTA/article/details/39910345)